In [11]:
import torch
from torch import nn
from torch import Tensor
from fvcore.nn import FlopCountAnalysis

In [ ]:
class TestModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc = nn.Linear(in_features=1000, out_features=10)
        self.conv = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=1)
        self.act = nn.ReLU()

    def forward(self, x) -> Tensor:
        conv_output_sparse = self.act(self.conv(x)).flatten(1).to_sparse()
        w_t_sparse = self.fc.weight.to_sparse()
        b_sparse = self.fc.bias.to_sparse()
        out = torch.sparse.mm(conv_output_sparse, w_t_sparse)
        out_bias = torch.sparse.addmm(out, b_sparse)
        # out_bias = torch.add(out, self.fc.bias)
        # out_bias = F.linear(conv_output, w_t_sparse, b_sparse)
        # out_bias = F.linear(conv_output.to_sparse(), self.fc.weight, self.fc.bias)
        # out_bias = torch.sparse.addmm(self.fc.bias, conv_output.to_sparse(), self.fc.weight.t())
        return out_bias

In [46]:
model = TestModel()
# model.fc.weight.data.fill_(0)
inputs = (torch.randn((1, 3, 10, 10)),)
flops = FlopCountAnalysis(model, inputs)

In [47]:
flops.total()

Unsupported operator aten::to_sparse encountered 1 time(s)
Unsupported operator aten::_sparse_addmm encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
fc


3000

In [ ]:
flops.total("fc")

In [ ]:
flops.by_operator()

In [ ]:
flops.by_module()

In [ ]:
flops.by_module_and_operator()